In [106]:
from dolfin import *
import numpy as np
import matplotlib.pylab as plt
import os
import datetime

def wave_eq(the_model                 , 
            num_intervals = 10        ,                                     
            out_name      = 'soln.pvd',
            build_dir     = ''):
    if( build_dir != '' ):
        if( os.path.exists(build_dir) == False ):
            os.system('mkdir %s'%(build_dir))
    
    #pass to local variable names
    c     = the_model.c
    mesh  = the_model.mesh
    dt    = the_model.dt
    T_0   = the_model.T_0
    T_1   = the_model.T_1
    delta = the_model.src
    V     = the_model.V
    N     = the_model.N
    
    # Time variables
    t = T_0
    T = T_1 - T_0

    #interval for showing plots
    plot_interval = np.floor(T / (dt * num_intervals))
    
    # Previous and current solution
    u1= interpolate(Constant(0.0), V)
    u0= interpolate(Constant(0.0), V)

    # Variational problem at each time
    u = TrialFunction(V)
    v = TestFunction(V)

    a = u*v*dx + dt*dt*c*c*inner(grad(u), grad(v))*dx
    L = 2*u1*v*dx-u0*v*dx

    bc = DirichletBC(V, 0, "on_boundary")
    A, b = assemble_system(a, L, bc)

    i_plot = 0
    u=Function(V)
    up=[]
    fid = File('solution.pvd')
    while t <= T_1:
        A, b = assemble_system(a, L, bc)
        delta_eval = delta(t)
        delta_eval.apply(b)
        solve(A, u.vector(), b)
        u0.assign(u1)
        u1.assign(u)
        t += dt

        # Reduce the range of the solution so that we can see the waves
        j = 0
        min_signal = 0.01
        for i in u.vector():
            i = min(.01, i)
            i = max(-.01, i)
            u.vector()[j] = i;
            j += 1

        if( (i_plot % plot_interval) == 0 ):
            #print('Hello about to plot...')
            print('About to plot')
            up.append(Function(V))
            up[len(up) - 1].assign(u)
            #plot(up, interactive=False, title=str('t = ' + str(t)))
            #plt.plot(mesh.coordinates(), u.vector())
            #fig.write_png(str('u-' + t + '.png'))
            
            
        i_plot += 1
    fid = File('%s%s'%(build_dir,out_name))
    for i in range(0,len(up)):
        t = dt * ( 1 + i * (T_1-T_0) / (dt * num_plots) )
        fid << up[i], t

In [107]:
class WaveModel:
    def __init__(self, mesh, space, deg=1, c=Constant(1.0), T_0=0, T_1=1, num_steps=100):
        self.mesh = mesh
        self.V    = FunctionSpace(mesh, space, deg)
        self.c    = c
        self.T_0  = T_0
        self.T_1  = T_1
        self.N    = num_steps
        self.dt   = (T_1 - T_0) / num_steps
        self.src  = lambda t : Constant(0.0)
    
    #source must be passed as a function of time, i.e. of type (t) -> ( R^2 -> R )
    def set_source(self, the_source):
        self.src = the_source
        
    #printing function for debugging
    def pretty_print(self):
        print('mesh = %s'%(self.mesh))
        print('space = %s'%(self.V))
        print('c = %s'%(self.c))
        print('T_0 = %s'%(self.T_0))
        print('T_1 = %s'%(self.T_1))
        print('N = %s'%(self.N))
        print('source = %s'%(self.src))
        
    #TODO: Modify so that the build_directory is passed as an argument, not dynamically calculated
    def go_forward(self,
                   num_intervals = 10        ,                                     
                   out_name      = 'soln.pvd',
                   build_dir     = ''):
        x = datetime.datetime.now()
        curr_time = '%s%s-%s:%s'%(x.strftime("%B"),x.strftime("%d"), \
                                    x.strftime("%H"), \
                                    x.strftime("%M"))
        build_dir = '%s/%s'%(build_dir, curr_time)
        os.system('mkdir %s'%(build_dir))
    
        #pass to local variable names
        c     = self.c
        mesh  = self.mesh
        dt    = self.dt
        T_0   = self.T_0
        T_1   = self.T_1
        delta = self.src
        V     = self.V
        N     = self.N
        
        # Time variables
        t = T_0
        T = T_1 - T_0

        #interval for showing plots
        plot_interval = np.floor(T / (dt * num_intervals))

        # Previous and current solution
        u1= interpolate(Constant(0.0), V)
        u0= interpolate(Constant(0.0), V)

        # Variational problem at each time
        u = TrialFunction(V)
        v = TestFunction(V)

        a = u*v*dx + dt*dt*c*c*inner(grad(u), grad(v))*dx
        L = 2*u1*v*dx-u0*v*dx

        bc = DirichletBC(V, 0, "on_boundary")
        A, b = assemble_system(a, L, bc)

        i_plot = 0
        u=Function(V)
        up=[]
        fid = File('solution.pvd')
        while t <= T_1:
            A, b = assemble_system(a, L, bc)
            delta_eval = delta(t)
            delta_eval.apply(b)
            solve(A, u.vector(), b)
            u0.assign(u1)
            u1.assign(u)
            t += dt

            # Reduce the range of the solution so that we can see the waves
            j = 0
            min_signal = 0.01
            for i in u.vector():
                i = min(.01, i)
                i = max(-.01, i)
                u.vector()[j] = i;
                j += 1

            if( (i_plot % plot_interval) == 0 ):
                #print('Hello about to plot...')
                print('About to plot')
                up.append(Function(V))
                up[len(up) - 1].assign(u)
                #plot(up, interactive=False, title=str('t = ' + str(t)))
                #plt.plot(mesh.coordinates(), u.vector())
                #fig.write_png(str('u-' + t + '.png'))


            i_plot += 1
        fid = File('%s/%s'%(build_dir,out_name))
        for i in range(0,len(up)):
            t = dt * ( 1 + i * (T_1-T_0) / (dt * num_plots) )
            fid << up[i], t
        os.system('cd %s'%(build_dir))
        os.system('sed -i "" \'s/f_[0-9]*/f/g\' \'.*vtu\'')
        os.system('sed -i "" \'s/UInt32/Int32/g\' \'.*vtu\'')
        
        

In [108]:
c         = Constant(5000.0)
mesh      = RectangleMesh(Point(-2, -2), Point(2, 2),80,80)
dt        = 4e-4
T_0       = 0
T_1       = 4e-3
num_plots = 100
V         = FunctionSpace(mesh, "Lagrange", 1)
delta     = lambda t : PointSource(V, Point(0.0, 0.0), sin(c * 10 * t))

the_model = WaveModel(mesh,"Lagrange",1)
#the_model.set_source(lambda t : PointSource(the_model.V, Point(0.0,0.0), sin(c * 10 * t)))
the_model.set_source(lambda t : PointSource(the_model.V,Point(0.0,0.0), 1.0))
the_model.pretty_print()
the_model.go_forward(build_dir='figures')


mesh = <dolfin.cpp.generation.RectangleMesh object at 0x129043d10>
space = FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 15216), FiniteElement('Lagrange', triangle, 1))
c = f_15214
T_0 = 0
T_1 = 1
N = 100
source = <function <lambda> at 0x12932a0d0>
About to plot
About to plot
About to plot
About to plot
About to plot
About to plot
About to plot
About to plot
About to plot
About to plot


In [81]:
x = the_model.src

In [82]:
x

<function __main__.<lambda>(t)>

ValueError: allocator<T>::allocate(size_t n) 'n' exceeds maximum supported size

In [84]:
the_model.pretty_print()

mesh = <dolfin.cpp.generation.RectangleMesh object at 0x1292f2a40>
space = FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 406), FiniteElement('Lagrange', triangle, 1))
c = f_370
T_0 = 0
T_1 = 1
N = 100
source = <function <lambda> at 0x128f7ddc0>


In [85]:
the_model.src

<function __main__.<lambda>(t)>